In [25]:
from gurobipy import *
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt
import itertools

In [26]:
output_file = 'testingfunction'

results = pd.read_table(f'../output/{output_file}.txt')
n = results[results["results"].str.contains("n")]
u = results[results["results"].str.contains("u")]
c = results[results["results"].str.contains("c")]
varn = n['results'].str.split('=').apply(lambda x: x[0])
valn = n['results'].str.split('=').apply(lambda x: x[1])
t = varn.str.split('n').apply(lambda x: ast.literal_eval(x[1])).apply(lambda x: x[0])
i = varn.str.split('n').apply(lambda x: ast.literal_eval(x[1])).apply(lambda x: x[1])
k = varn.str.split('n').apply(lambda x: ast.literal_eval(x[1])).apply(lambda x: x[2])
amount = valn
specificn = pd.DataFrame(np.array([t,i,k,amount]).T).reset_index(drop=True)
specificn.columns = ['t','i', 'k', 'amount']
specificn['name'] = 'n'

varu = u['results'].str.split('=').apply(lambda x: x[0])
valu = u['results'].str.split('=').apply(lambda x: x[1])
t = varu.str.split('u').apply(lambda x: ast.literal_eval(x[1])).apply(lambda x: x[0])
i = varu.str.split('u').apply(lambda x: ast.literal_eval(x[1])).apply(lambda x: x[1])
j = varu.str.split('u').apply(lambda x: ast.literal_eval(x[1])).apply(lambda x: x[2])
k = varu.str.split('u').apply(lambda x: ast.literal_eval(x[1])).apply(lambda x: x[3])
amount = valu
specificu = pd.DataFrame(np.array([t,i,j,k,amount]).T).reset_index(drop=True)
specificu.columns = ['t','i', 'j', 'k', 'amount']
specificu['name'] = 'u'

varc = c['results'].str.split('=').apply(lambda x: x[0])
valc = c['results'].str.split('=').apply(lambda x: x[1])
t = varc.str.split('c').apply(lambda x: ast.literal_eval(x[1])).apply(lambda x: x[0])
i = varc.str.split('c').apply(lambda x: ast.literal_eval(x[1])).apply(lambda x: x[1])
x = varc.str.split('c').apply(lambda x: ast.literal_eval(x[1])).apply(lambda x: x[2])
y = varc.str.split('c').apply(lambda x: ast.literal_eval(x[1])).apply(lambda x: x[3])
amount = valc
specificc = pd.DataFrame(np.array([t,i,x,y,amount]).T).reset_index(drop=True)
specificc.columns = ['t','i', 'x', 'y', 'amount']
specificc['name'] = 'c'
specificn['amount'] = specificn['amount'].astype(float)
specificu['amount'] = specificu['amount'].astype(float)
specificc['amount'] = specificc['amount'].astype(float)


In [27]:
specificu01 = specificu[specificu['i'] == 0]
specificu10 = specificu[specificu['i'] == 1]

In [28]:
# soc_level = {
#     0: 20,
#     1: 30,
#     2: 40,
#     3: 50,
#     4: 60,
#     5: 70,
#     6: 80,
#     7: 90,
#     8: 100
# }

# soc_level = {
#     0: 20, 1: 25, 2: 30, 3: 35, 4: 40, 
#     5: 45, 6: 50, 7: 55, 8: 60, 9: 65, 
#     10: 70, 11: 75, 12: 80, 13: 85, 
#     14: 90, 15: 95, 16: 100
# }

soc_level = {
    0: 20, 1: 22.5, 2: 25, 3: 27.5, 4: 30,
    5: 32.5, 6: 35, 7: 37.5, 8: 40, 9: 42.5,
    10: 45, 11: 47.5, 12: 50, 13: 52.5,
    14: 55, 15: 57.5, 16: 60, 17: 62.5,
    18: 65, 19: 67.5, 20: 70, 21: 72.5,
    22: 75, 23: 77.5, 24: 80, 25: 82.5,
    26: 85, 27: 87.5, 28: 90, 29: 92.5,
    30: 95, 31: 97.5, 32: 100
}

In [29]:
# gamma = [3.237872699, 3.729470167, 4.404786588, 5.379957014, 6.913363091,
#         9.685271742, 16.30528373, 71.41103553]

# _gamma = [1.567183013,1.670689686,1.79349788,1.935972287,2.103057098,
#          2.30172949,2.541890384,2.83806663,3.212473781,3.70088931,
#          4.364896382,5.32037536,6.814736187,9.490547548,15.74119426,55.66984127]
# gamma = np.array(_gamma) / 5

_gamma = np.array([
0.0129,0.0133,0.0137,0.0142,0.0147,
0.0153,0.0158,0.0166,0.0172,0.018,
0.0188,0.0197,0.0207,0.0219,0.0231,
0.0245,0.026,0.0278,0.03,0.0323,
0.0351,0.0384,0.0423,0.0472,0.0536,
0.0617,0.0726,0.0887,0.1136,0.1582,
0.2622,0.9278,
])*60

gamma = np.array(_gamma) / 2.5

gamma = np.ceil(gamma)

In [30]:
def rename_columns(df):
    df.columns = ['t', 'num_flights', 'aircraft_pushback_time']
    return df

def convert_timestep_to_sec(df, timestep=5):
    df['aircraft_pushback_time'] = df.loc[:, 't'] * timestep * 60
    return df

def replicate_rows(df, column_name):
    return df.loc[df.index.repeat(df[column_name])].reset_index(drop=True)

def reorganize_df(df, timestep=5):
    df = convert_timestep_to_sec(df, timestep)
    df = replicate_rows(df, 'amount')
    # df = rename_columns(df)
    return df

def get_charge_time(row):
    start = max(row['x']-1, 0)
    end = row['y']
    return sum(gamma[start:end])*60*5

def create_location(df, location_col):
    df['location'] = df.groupby(location_col).cumcount() + 11
    df['location'] = df[location_col] + '_PARK' + df['location'].astype(str)
    df = none_location(df)
    return df

def create_location(df, location_col):
    # Start a dictionary to keep track of counts for each location
    counts_dict = {}

    # Apply a function to each row to generate the location name
    def make_location(row):
        location = row[location_col]
        # If the location is in the dictionary, use the next number, otherwise start from 1
        if location in counts_dict:
            counts_dict[location] += 1
        else:
            counts_dict[location] = 1

        # Create the location name and return it
        location_name = f"{location}_PARK1{counts_dict[location]}"
        return location_name

    df['location'] = df.apply(make_location, axis=1)
    df = none_location(df)
    return df





# add location if the charging_start_time is 0, otherwise add None
def none_location(df):
    df['location'] = np.where(df['charging_start_time'] == 0, df['location'], None)
    return df

def count_locations(df):
    dtla_park_count = df['location'].str.contains('DTLA_PARK').sum()
    lax_park_count = df['location'].str.contains('LAX_PARK').sum()
    return dtla_park_count, lax_park_count

def seconds_to_hms(seconds):
    hours = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
    return "%02d:%02d:%02d" % (hours, minutes, seconds)

In [31]:
specificc

,t,i,x,y,amount,name
0,0,0,2,3,1.0,c
1,0,0,5,12,1.0,c
2,0,0,11,13,1.0,c
3,0,0,15,16,1.0,c
4,0,1,6,7,2.0,c
...,...,...,...,...,...,...
475,290,0,2,3,1.0,c
476,290,0,5,12,1.0,c
477,290,0,11,13,1.0,c
478,290,0,15,16,1.0,c


In [32]:
df_c = specificc
df_c = replicate_rows(df_c, 'amount')
df_c['charging_start_time'] = df_c['t'] * 5 * 60
# df_c['charging_time'] = df_c.apply(get_charge_time, axis=1)
df_c['i'] = df_c['i'].apply(lambda x: 'LAX' if x == 0 else 'DTLA')
df_c.rename(columns={'amount': 'num_aircraft', 
                     'x': 'init_soc',
                     'y': 'target_soc',
                     'i': 'vertiport_id'}, inplace=True)
df_c['init_soc'] = df_c['init_soc'].apply(lambda x: soc_level[x])
df_c['target_soc'] = df_c['target_soc'].apply(lambda x: soc_level[x])
# Duplicate rows based on num_aircraft column
# df_c = df_c.loc[df_c.index.repeat(df_c.num_aircraft)].reset_index(drop=True)
df_c.drop(['t','name', 'num_aircraft'], axis=1, inplace=True)
df_c['process'] = 'CHARGING'
df_c

,vertiport_id,init_soc,target_soc,charging_start_time,process
0,LAX,25.0,27.5,0,CHARGING
1,LAX,32.5,50.0,0,CHARGING
2,LAX,47.5,52.5,0,CHARGING
3,LAX,57.5,60.0,0,CHARGING
4,DTLA,35.0,37.5,0,CHARGING
...,...,...,...,...,...
539,LAX,32.5,50.0,87000,CHARGING
540,LAX,47.5,52.5,87000,CHARGING
541,LAX,57.5,60.0,87000,CHARGING
542,DTLA,35.0,37.5,87000,CHARGING


In [33]:
def get_idling_aircraft(df):
    df = reorganize_df(df)
    # df = df.loc[df.index.repeat(df.num_flights)].reset_index(drop=True)
    df['vertiport_id'] = df['i'].apply(lambda x: 'LAX' if x == 0 else 'DTLA')
    df = df[df['aircraft_pushback_time'] == 0]
    df.drop(['t', 'i', 'name', 'amount', 'aircraft_pushback_time'], axis=1, inplace=True)
    df.rename(columns={'k': 'soc'}, inplace=True)
    df['init_soc'] = df['soc'].apply(lambda x: soc_level[x])
    df['target_soc'] = df['soc'].apply(lambda x: soc_level[x])
    df['charging_start_time'] = 0
    df.drop('soc', axis=1, inplace=True)
    df['process'] = 'PARKING' 
    return df

In [34]:
idle_aircraft = get_idling_aircraft(specificn)
idle_aircraft

,vertiport_id,init_soc,target_soc,charging_start_time,process
0,LAX,52.5,52.5,0,PARKING
1,LAX,57.5,57.5,0,PARKING
2,DTLA,25.0,25.0,0,PARKING
3,DTLA,25.0,25.0,0,PARKING
4,DTLA,27.5,27.5,0,PARKING
5,DTLA,27.5,27.5,0,PARKING
6,DTLA,45.0,45.0,0,PARKING


In [35]:
init_states = pd.concat([df_c, idle_aircraft]).sort_values(['charging_start_time']).reset_index(drop=True)
init_states = create_location(init_states, 'vertiport_id')
init_states['charging_start_time_datetime'] = init_states['charging_start_time'].apply(seconds_to_hms)
init_states.head(20)

,vertiport_id,init_soc,target_soc,charging_start_time,process,location,charging_start_time_datetime
0,LAX,25.0,27.5,0,CHARGING,LAX_PARK11,00:00:00
1,DTLA,27.5,27.5,0,PARKING,DTLA_PARK11,00:00:00
2,DTLA,25.0,25.0,0,PARKING,DTLA_PARK12,00:00:00
3,DTLA,25.0,25.0,0,PARKING,DTLA_PARK13,00:00:00
4,LAX,57.5,57.5,0,PARKING,LAX_PARK12,00:00:00
5,LAX,52.5,52.5,0,PARKING,LAX_PARK13,00:00:00
6,DTLA,27.5,27.5,0,PARKING,DTLA_PARK14,00:00:00
7,DTLA,45.0,45.0,0,PARKING,DTLA_PARK15,00:00:00
8,DTLA,35.0,37.5,0,CHARGING,DTLA_PARK16,00:00:00
9,DTLA,35.0,37.5,0,CHARGING,DTLA_PARK17,00:00:00


In [36]:
init_states.to_csv('../output/charge_schedule.csv', index=False)

In [37]:
dtla_park_count, lax_park_count = count_locations(init_states)
print('DTLA PARK COUNT: ', dtla_park_count, 'LAX PARK COUNT: ', lax_park_count)

DTLA PARK COUNT:  7 LAX PARK COUNT:  6


In [38]:
def create_flight_schedule(df, origin, destination):
    df = reorganize_df(df)
    # df = df.loc[df.index.repeat(df.num_flights)].reset_index(drop=True)
    df['origin_vertiport_id'] = origin
    df['destination_vertiport_id'] = destination
    # df['aircraft_pushback_time'] = df['aircraft_pushback_time'] + df.groupby('aircraft_pushback_time').cumcount() * 30
    df.drop(['t', 'i', 'j', 'name', 'amount'], axis=1, inplace=True)
    df.rename(columns={'k': 'soc'}, inplace=True)
    df['soc'] = df['soc'].apply(lambda x: soc_level[x])
    df = df[['origin_vertiport_id', 'destination_vertiport_id', 'aircraft_pushback_time', 'soc']]
    df['tail_number'] = None
    df['aircraft_pushback_datetime'] = df['aircraft_pushback_time'].apply(seconds_to_hms)
    return df

In [39]:
lax_dtla = create_flight_schedule(specificu01, origin='LAX', destination='DTLA')
dtla_lax = create_flight_schedule(specificu10, origin='DTLA', destination='LAX')

/var/folders/f3/kcmw8dgs2yj5rlxtpqsq4nbc0000gn/T/ipykernel_70163/3671448927.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['aircraft_pushback_time'] = df.loc[:, 't'] * timestep * 60
/var/folders/f3/kcmw8dgs2yj5rlxtpqsq4nbc0000gn/T/ipykernel_70163/3671448927.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['aircraft_pushback_time'] = df.loc[:, 't'] * timestep * 60


In [40]:
flight_schedule = pd.concat([lax_dtla, dtla_lax]).sort_values('aircraft_pushback_time').reset_index(drop=True)
flight_schedule['aircraft_pushback_time'] = flight_schedule['aircraft_pushback_time']
flight_schedule

,origin_vertiport_id,destination_vertiport_id,aircraft_pushback_time,soc,tail_number,aircraft_pushback_datetime
0,LAX,DTLA,300,52.5,None,00:05:00
1,DTLA,LAX,300,37.5,None,00:05:00
2,DTLA,LAX,300,45.0,None,00:05:00
3,DTLA,LAX,300,37.5,None,00:05:00
4,DTLA,LAX,300,27.5,None,00:05:00
...,...,...,...,...,...,...
877,LAX,DTLA,86400,32.5,None,24:00:00
878,LAX,DTLA,86400,30.0,None,24:00:00
879,LAX,DTLA,86400,30.0,None,24:00:00
880,LAX,DTLA,86400,32.5,None,24:00:00


In [41]:
flight_schedule.to_csv('../output/flight_schedule.csv', index=False)